## Walmart Store Sales Prediction

Objective: The objective is predicting store sales using historical markdown data.

In [1]:
import findspark
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
findspark.init()

In [3]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from pyspark import SparkFiles


import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import statsmodels.api as sm

from datetime import datetime

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Getting Data by PySpark

In [4]:
spark = SparkSession.builder.appName("myApp").config("spark.mongodb.input.uri","mongodb://localhost:27017/project.walmart?readPreference=primaryPreferred").config("spark.mongodb.output.uri","mongodb://localhost:27017/project.walmart").config("spark.jars.packages","org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").getOrCreate()

In [6]:
url="https://raw.githubusercontent.com/kirtigupta10007/Walmart-Store-Sales-Forecasting/master/data/features.csv"
spark.sparkContext.addFile(url)
SparkFiles.get("features.csv")
features=spark.read.csv("file:///"+SparkFiles.get("features.csv"), header=True, inferSchema= True)

In [7]:
url="https://raw.githubusercontent.com/kirtigupta10007/Walmart-Store-Sales-Forecasting/master/data/stores.csv"
spark.sparkContext.addFile(url)
SparkFiles.get("stores.csv")
stores=spark.read.csv("file:///"+SparkFiles.get("stores.csv"), header=True, inferSchema= True)

In [8]:
url="https://raw.githubusercontent.com/kirtigupta10007/Walmart-Store-Sales-Forecasting/master/data/train.csv"
spark.sparkContext.addFile(url)
SparkFiles.get("train.csv")
train=spark.read.csv("file:///"+SparkFiles.get("train.csv"), header=True, inferSchema= True)

In [9]:
url="https://raw.githubusercontent.com/kirtigupta10007/Walmart-Store-Sales-Forecasting/master/data/test.csv"
spark.sparkContext.addFile(url)
SparkFiles.get("test.csv")
test=spark.read.csv("file:///"+SparkFiles.get("test.csv"), header=True, inferSchema= True)

In [10]:
train.show(2)

+-----+----+----------+------------+---------+
|Store|Dept|      Date|Weekly_Sales|IsHoliday|
+-----+----+----------+------------+---------+
|    1|   1|2010-02-05|     24924.5|    false|
|    1|   1|2010-02-12|    46039.49|     true|
+-----+----+----------+------------+---------+
only showing top 2 rows



In [11]:
test.show(2)

+-----+----+----------+---------+
|Store|Dept|      Date|IsHoliday|
+-----+----+----------+---------+
|    1|   1|2012-11-02|    false|
|    1|   1|2012-11-09|    false|
+-----+----+----------+---------+
only showing top 2 rows



In [12]:
features.show(2)

+-----+----------+-----------+----------+---------+---------+---------+---------+---------+-----------+------------+---------+
|Store|      Date|Temperature|Fuel_Price|MarkDown1|MarkDown2|MarkDown3|MarkDown4|MarkDown5|        CPI|Unemployment|IsHoliday|
+-----+----------+-----------+----------+---------+---------+---------+---------+---------+-----------+------------+---------+
|    1|2010-02-05|      42.31|     2.572|       NA|       NA|       NA|       NA|       NA|211.0963582|       8.106|    false|
|    1|2010-02-12|      38.51|     2.548|       NA|       NA|       NA|       NA|       NA|211.2421698|       8.106|     true|
+-----+----------+-----------+----------+---------+---------+---------+---------+---------+-----------+------------+---------+
only showing top 2 rows



In [13]:
stores.show(2)

+-----+----+------+
|Store|Type|  Size|
+-----+----+------+
|    1|   A|151315|
|    2|   A|202307|
+-----+----+------+
only showing top 2 rows



Checking Null Values

In [14]:
chcekNullValues = {col:features.filter(features[col].isNull()).count() for col in features.columns}
print(chcekNullValues)

{'Store': 0, 'Date': 0, 'Temperature': 0, 'Fuel_Price': 0, 'MarkDown1': 0, 'MarkDown2': 0, 'MarkDown3': 0, 'MarkDown4': 0, 'MarkDown5': 0, 'CPI': 0, 'Unemployment': 0, 'IsHoliday': 0}


In [15]:
chcekNullValues = {col:stores.filter(stores[col].isNull()).count() for col in stores.columns}
print(chcekNullValues)

{'Store': 0, 'Type': 0, 'Size': 0}


In [16]:
chcekNullValues = {col:train.filter(train[col].isNull()).count() for col in train.columns}
print(chcekNullValues)

{'Store': 0, 'Dept': 0, 'Date': 0, 'Weekly_Sales': 0, 'IsHoliday': 0}


In [17]:
chcekNullValues = {col:test.filter(test[col].isNull()).count() for col in test.columns}
print(chcekNullValues)

{'Store': 0, 'Dept': 0, 'Date': 0, 'IsHoliday': 0}


### Data Pre-Processing 

    Merging the DataSet 
      -(train+Store+Feature) 
      -(test+Store+Feature)

In [18]:
train_bt=train.join(stores, ["Store"])
train = train_bt.join(features, on=['Store', 'Date','IsHoliday'], how='inner')

In [19]:
test_bt=test.join(stores,["Store"])
test = test_bt.join(features, on=['Store', 'Date','IsHoliday'], how='inner')

In [20]:
train.show(2)

+-----+----------+---------+----+------------+----+------+-----------+----------+---------+---------+---------+---------+---------+-----------+------------+
|Store|      Date|IsHoliday|Dept|Weekly_Sales|Type|  Size|Temperature|Fuel_Price|MarkDown1|MarkDown2|MarkDown3|MarkDown4|MarkDown5|        CPI|Unemployment|
+-----+----------+---------+----+------------+----+------+-----------+----------+---------+---------+---------+---------+---------+-----------+------------+
|    1|2010-02-05|    false|   1|     24924.5|   A|151315|      42.31|     2.572|       NA|       NA|       NA|       NA|       NA|211.0963582|       8.106|
|    1|2010-02-12|     true|   1|    46039.49|   A|151315|      38.51|     2.548|       NA|       NA|       NA|       NA|       NA|211.2421698|       8.106|
+-----+----------+---------+----+------------+----+------+-----------+----------+---------+---------+---------+---------+---------+-----------+------------+
only showing top 2 rows



In [21]:
test.show(2)

+-----+----------+---------+----+----+------+-----------+----------+---------+---------+---------+---------+---------+-----------+------------+
|Store|      Date|IsHoliday|Dept|Type|  Size|Temperature|Fuel_Price|MarkDown1|MarkDown2|MarkDown3|MarkDown4|MarkDown5|        CPI|Unemployment|
+-----+----------+---------+----+----+------+-----------+----------+---------+---------+---------+---------+---------+-----------+------------+
|    1|2012-11-02|    false|   1|   A|151315|      55.32|     3.386|  6766.44|   5147.7|    50.82|   3639.9|  2737.42|223.4627793|       6.573|
|    1|2012-11-09|    false|   1|   A|151315|      61.24|     3.314| 11421.32|  3370.89|    40.28|  4646.79|  6154.16|223.4813073|       6.573|
+-----+----------+---------+----+----+------+-----------+----------+---------+---------+---------+---------+---------+-----------+------------+
only showing top 2 rows



In [22]:
print(train.printSchema())
print("*****************************************")
print(test.printSchema())

root
 |-- Store: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- IsHoliday: boolean (nullable = true)
 |-- Dept: integer (nullable = true)
 |-- Weekly_Sales: double (nullable = true)
 |-- Type: string (nullable = true)
 |-- Size: integer (nullable = true)
 |-- Temperature: double (nullable = true)
 |-- Fuel_Price: double (nullable = true)
 |-- MarkDown1: string (nullable = true)
 |-- MarkDown2: string (nullable = true)
 |-- MarkDown3: string (nullable = true)
 |-- MarkDown4: string (nullable = true)
 |-- MarkDown5: string (nullable = true)
 |-- CPI: string (nullable = true)
 |-- Unemployment: string (nullable = true)

None
*****************************************
root
 |-- Store: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- IsHoliday: boolean (nullable = true)
 |-- Dept: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Size: integer (nullable = true)
 |-- Temperature: double (nullable = true)
 |-- Fuel_Price: double (nullable 

#### Saving Data in MongoDB

In [ ]:
ml = spark.read.format("mongo").option("uri","mongodb://localhost:27017/walmart.data_train").load()
train.write.format("mongo").option("uri","mongodb://localhost:27017/walmart.data_train").save()

In [ ]:
ml_test = spark.read.format("mongo").option("uri","mongodb://localhost:27017/walmart.data_test").load()
test.write.format("mongo").option("uri","mongodb://localhost:27017/walmart.data_test").save()

### Getting Data from MongoDB to Pandas

In [23]:
import pymongo
from pymongo import MongoClient

In [24]:
client = MongoClient('localhost',27017)
db = client.walmart
train = db.data_train
train = pd.DataFrame(list(train.find())).drop(['_id'],axis=1)

In [25]:
client = MongoClient('localhost',27017)
db_test = client.walmart
data_test = db.data_test
test = pd.DataFrame(list(data_test.find())).drop(['_id'],axis=1)

In [26]:
print(train.info())
print ("*****************************************")
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Date          421570 non-null  object 
 2   IsHoliday     421570 non-null  bool   
 3   Dept          421570 non-null  int64  
 4   Weekly_Sales  421570 non-null  float64
 5   Type          421570 non-null  object 
 6   Size          421570 non-null  int64  
 7   Temperature   421570 non-null  float64
 8   Fuel_Price    421570 non-null  float64
 9   MarkDown1     421570 non-null  object 
 10  MarkDown2     421570 non-null  object 
 11  MarkDown3     421570 non-null  object 
 12  MarkDown4     421570 non-null  object 
 13  MarkDown5     421570 non-null  object 
 14  CPI           421570 non-null  object 
 15  Unemployment  421570 non-null  object 
dtypes: bool(1), float64(3), int64(3), object(9)
memory usage: 48.6+ MB
None
********************

### Cleaning Data

Other Missing Value Treatment like Markdown
Imputing it with Zero(No Markdown)

In [27]:
train['MarkDown1'] = train['MarkDown1'].str.replace('NA','0').astype(np.float64)
train['MarkDown2'] = train['MarkDown2'].str.replace('NA','0').astype(np.float64)
train['MarkDown3'] = train['MarkDown3'].str.replace('NA','0').astype(np.float64)
train['MarkDown4'] = train['MarkDown4'].str.replace('NA','0').astype(np.float64)
train['MarkDown5'] = train['MarkDown5'].str.replace('NA','0').astype(np.float64)
train['CPI'] = train['CPI'].astype(np.float64)
train['Unemployment'] = train['Unemployment'].astype(np.float64)

In [28]:
train.head(5)

,Store,Date,IsHoliday,Dept,Weekly_Sales,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,2010-02-05,False,1,24924.50,A,151315,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106
1,1,2010-02-12,True,1,46039.49,A,151315,38.51,2.548,0.0,0.0,0.0,0.0,0.0,211.242170,8.106
2,1,2010-02-19,False,1,41595.55,A,151315,39.93,2.514,0.0,0.0,0.0,0.0,0.0,211.289143,8.106
3,1,2010-02-26,False,1,19403.54,A,151315,46.63,2.561,0.0,0.0,0.0,0.0,0.0,211.319643,8.106
4,1,2010-03-05,False,1,21827.90,A,151315,46.50,2.625,0.0,0.0,0.0,0.0,0.0,211.350143,8.106


In [29]:
train.dtypes

Store             int64
Date             object
IsHoliday          bool
Dept              int64
Weekly_Sales    float64
Type             object
Size              int64
Temperature     float64
Fuel_Price      float64
MarkDown1       float64
MarkDown2       float64
MarkDown3       float64
MarkDown4       float64
MarkDown5       float64
CPI             float64
Unemployment    float64
dtype: object

In [30]:
test['MarkDown1'] = test['MarkDown1'].str.replace('NA','0').astype(np.float64)
test['MarkDown2'] = test['MarkDown2'].str.replace('NA','0').astype(np.float64)
test['MarkDown3'] = test['MarkDown3'].str.replace('NA','0').astype(np.float64)
test['MarkDown4'] = test['MarkDown4'].str.replace('NA','0').astype(np.float64)
test['MarkDown5'] = test['MarkDown5'].str.replace('NA','0').astype(np.float64)
test['CPI'] = test['CPI'].str.replace('NA','0').astype(np.float64)
test['Unemployment'] = test['Unemployment'].str.replace('NA','0').astype(np.float64)

In [31]:
test.head(5)

,Store,Date,IsHoliday,Dept,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,2012-11-02,False,1,A,151315,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,6.573
1,1,2012-11-09,False,1,A,151315,61.24,3.314,11421.32,3370.89,40.28,4646.79,6154.16,223.481307,6.573
2,1,2012-11-16,False,1,A,151315,52.92,3.252,9696.28,292.10,103.78,1133.15,6612.69,223.512911,6.573
3,1,2012-11-23,True,1,A,151315,56.23,3.211,883.59,4.17,74910.32,209.91,303.32,223.561947,6.573
4,1,2012-11-30,False,1,A,151315,52.34,3.207,2460.03,0.00,3838.35,150.57,6966.34,223.610984,6.573


In [32]:
test.dtypes

Store             int64
Date             object
IsHoliday          bool
Dept              int64
Type             object
Size              int64
Temperature     float64
Fuel_Price      float64
MarkDown1       float64
MarkDown2       float64
MarkDown3       float64
MarkDown4       float64
MarkDown5       float64
CPI             float64
Unemployment    float64
dtype: object

#### Data Description

##### 1. Training Data

In [33]:
numeric_var_train=[key for key in dict(train.dtypes) if dict(train.dtypes)[key] in ['float64', 'int64', 'float32', 'int32']]
cat_var_train=[key for key in dict(train.dtypes) if dict(train.dtypes)[key] in ['object']]
# Train Numerical Data
train_num=train[numeric_var_train]

# Train Categorical Data
train_cat=train[cat_var_train]
print(numeric_var_train)
print(cat_var_train)

['Store', 'Dept', 'Weekly_Sales', 'Size', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment']
['Date', 'Type']


In [37]:
!pip install pyxll-jupyter

In [36]:
# Creating Data audit Report
# Use a general function that returns multiple values    
def var_summary(x):
    return pd.Series([x.count(), x.isnull().sum(), x.sum(), x.mean(), x.median(),  x.std(), x.var(), x.min(), x.dropna().quantile(0.01), x.dropna().quantile(0.05),x.dropna().quantile(0.10),x.dropna().quantile(0.25),x.dropna().quantile(0.50),x.dropna().quantile(0.75), x.dropna().quantile(0.90),x.dropna().quantile(0.95), x.dropna().quantile(0.99),x.max()], 
                  index=['N', 'NMISS', 'SUM', 'MEAN','MEDIAN', 'STD', 'VAR', 'MIN', 'P1' , 'P5' ,'P10' ,'P25' ,'P50' ,'P75' ,'P90' ,'P95' ,'P99' ,'MAX'])
 
num_summary=train_num.apply(lambda x: var_summary(x)).T
num_summary.to_excel(writer,'Numeric_variable Summary',index=True)
num_summary

NameError: name 'writer' is not defined

In [ ]:
def cat_summary(x):
    return pd.Series([x.count(), x.isnull().sum(), x.value_counts()], 
                  index=['N', 'NMISS', 'ColumnsNames'])

cat_summary=train_cat.apply(lambda x: cat_summary(x))
cat_summary

##### 2. Testing Data

In [ ]:
numeric_var_test=[key for key in dict(test.dtypes) if dict(test.dtypes)[key] in ['float64', 'int64', 'float32', 'int32']]
cat_var_test=[key for key in dict(test.dtypes) if dict(test.dtypes)[key] in ['object']]
# Train Numerical Data
test_num=test[numeric_var_test]

# Train Categorical Data
test_cat=test[cat_var_test]
print numeric_var_test
print cat_var_test

In [ ]:
num_summary=test_num.apply(lambda x: var_summary(x)).T
#num_summary.to_excel(writer,'Numeric_variable Summary',index=True)
num_summary.head()

In [ ]:
def cat_summary(x):
    return pd.Series([x.count(), x.isnull().sum(), x.value_counts()], 
                  index=['N', 'NMISS', 'ColumnsNames'])

cat_summary=test_cat.apply(lambda x: cat_summary(x))
cat_summary

###### Correlation

In [ ]:
train_corr=pd.DataFrame(train.corr())
train_corr.to_excel(writer,'Train_Data Corr',index=True)
train_corr.head()

In [ ]:
test_corr=pd.DataFrame(test.corr())
#test_corr.to_excel(writer,'Test_Data Corr',index=True)
test_corr.head()

In [ ]:
# visualize correlation matrix in Seaborn using a heatmap
sns.heatmap(train.corr())

<big> -Markdowns are Highly Correlated.

In [ ]:
# visualize correlation matrix in Seaborn using a heatmap
sns.heatmap(test.corr())

<big> -Markdown 4 is  Highly Correlated to Markdown 1

___

### Data Exploratory Analysis

In [ ]:
train['Store'].value_counts(normalize=True).plot(kind = 'bar',fig=(4,5))

<big> -The above graph shows that There are more number of Store 13 and very less number of Store 36

###### Store Size vs Sales

In [ ]:
train.plot(kind='line', x='Weekly_Sales', y='Store', alpha=0.5)

<big> From this plot, we notice that Store 10 has the highest Sales

###### Sales vs Type

In [ ]:
#tips = sns.load_dataset('train')
sns.barplot(x=train["Weekly_Sales"],y=train["Type"])

<big> From this plot, we notice that 'type C' stores have fewer sales and 'type A' stores have more sales in general.

###### Sales vs Deptartment

In [ ]:
train.plot(kind='line', x='Dept', y='Weekly_Sales', alpha=1.5,fig=(4,5))

<big> From this plot, we notice Deptartment with the highest sales lies between Dept 60 and 80

____

#### Missing Value Treatment

In [ ]:
print train.isnull().sum()
print "*"*30
print test.isnull().sum()

    Imputing it with its mean

In [ ]:
test['CPI']=test.groupby(['Dept'])['CPI'].transform(lambda x: x.fillna(x.mean()))
test['Unemployment']=test.groupby(['Dept'])['Unemployment'].transform(lambda x: x.fillna(x.mean()))

###### Outlier Treatment

In [ ]:
train.Weekly_Sales=np.where(train.Weekly_Sales>100000, 100000,train.Weekly_Sales)

In [ ]:
train.Weekly_Sales.plot.hist(bins=25)

_______________

### Feature Extraction
In this section, we select the appropriate features to train our classifier. Here, we create new features based on existing features. We also convert categorical features into numeric form.

###### Date Feature

In [ ]:
train.info()

In [ ]:
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

In [ ]:
# Extract date features
train['Date_dayofweek'] =train['Date'].dt.dayofweek
train['Date_month'] =train['Date'].dt.month 
train['Date_year'] =train['Date'].dt.year
train['Date_day'] =train['Date'].dt.day 

test['Date_dayofweek'] =test['Date'].dt.dayofweek
test['Date_month'] =test['Date'].dt.month 
test['Date_year'] =test['Date'].dt.year
test['Date_day'] =test['Date'].dt.day 

In [ ]:
print train.Type.value_counts()
print "*"*30
print test.Type.value_counts()

In [ ]:
print train.IsHoliday.value_counts()
print "*"*30
print test.IsHoliday.value_counts()

In [ ]:
train_test_data = [train, test]

    Converting Categorical Variable 'Type' into Numerical Variable 
    For A=1 , B=2, C=3

In [ ]:
type_mapping = {"A": 1, "B": 2, "C": 3}
for dataset in train_test_data:
    dataset['Type'] = dataset['Type'].map(type_mapping)

    Converting Categorical Variable 'IsHoliday' into Numerical Variable 

In [ ]:
type_mapping = {False: 0, True: 1}
for dataset in train_test_data:
    dataset['IsHoliday'] = dataset['IsHoliday'].map(type_mapping)

    Creating Extra Holiday Variable.
    If that week comes under extra holiday then 1(=Yes) else 2(=No)

Making New Holiday Variable Based on Given Data....

In [ ]:
train['Super_Bowl'] = np.where((train['Date']==datetime(2010, 2, 12)) | (train['Date']==datetime(2011, 2, 11)) | (train['Date']==datetime(2012, 2, 10)) | (train['Date']==datetime(2013, 2, 8)),1,0)
train['Labour_Day'] = np.where((train['Date']==datetime(2010, 9, 10)) | (train['Date']==datetime(2011, 9, 9)) | (train['Date']==datetime(2012, 9, 7)) | (train['Date']==datetime(2013, 9, 6)),1,0)
train['Thanksgiving'] = np.where((train['Date']==datetime(2010, 11, 26)) | (train['Date']==datetime(2011, 11, 25)) | (train['Date']==datetime(2012, 11, 23)) | (train['Date']==datetime(2013, 11, 29)),1,0)
train['Christmas'] = np.where((train['Date']==datetime(2010, 12, 31)) | (train['Date']==datetime(2011, 12, 30)) | (train['Date']==datetime(2012, 12, 28)) | (train['Date']==datetime(2013, 12, 27)),1,0)
#........................................................................
test['Super_Bowl'] = np.where((test['Date']==datetime(2010, 2, 12)) | (test['Date']==datetime(2011, 2, 11)) | (test['Date']==datetime(2012, 2, 10)) | (test['Date']==datetime(2013, 2, 8)),1,0)
test['Labour_Day'] = np.where((test['Date']==datetime(2010, 9, 10)) | (test['Date']==datetime(2011, 9, 9)) | (test['Date']==datetime(2012, 9, 7)) | (test['Date']==datetime(2013, 9, 6)),1,0)
test['Thanksgiving'] = np.where((test['Date']==datetime(2010, 11, 26)) | (test['Date']==datetime(2011, 11, 25)) | (test['Date']==datetime(2012, 11, 23)) | (test['Date']==datetime(2013, 11, 29)),1,0)
test['Christmas'] = np.where((test['Date']==datetime(2010, 12, 31)) | (test['Date']==datetime(2011, 12, 30)) | (test['Date']==datetime(2012, 12, 28)) | (test['Date']==datetime(2013, 12, 27)),1,0)


In [ ]:
# Altering the isHoliday value depending on these new holidays...
train['IsHoliday']=train['IsHoliday']|train['Super_Bowl']|train['Labour_Day']|train['Thanksgiving']|train['Christmas']
test['IsHoliday']=test['IsHoliday']|test['Super_Bowl']|test['Labour_Day']|test['Thanksgiving']|test['Christmas']

In [ ]:
print train.Christmas.value_counts()
print train.Super_Bowl.value_counts()
print train.Thanksgiving.value_counts()
print train.Labour_Day.value_counts()

In [ ]:
print test.Christmas.value_counts()
print test.Super_Bowl.value_counts()
print test.Thanksgiving.value_counts()
print test.Labour_Day.value_counts()

In [ ]:
# Since we have Imputed IsHoliday according to Extra holidays..These extra holiday variable has redundant..
# Droping the Extra holiday variables because its redundant..
dp=['Super_Bowl','Labour_Day','Thanksgiving','Christmas']
train.drop(dp,axis=1,inplace=True)
test.drop(dp,axis=1,inplace=True)

### Feature Selection

In [ ]:
train.info()

####    Droping irrevelent variable:
    -Since we have imputed markdown variables therefore we will not be removing the all markdown variables.
    -Removing MarkDown5 because its Highly Skewed

In [ ]:
features_drop=['Unemployment','CPI','MarkDown5']
train=train.drop(features_drop, axis=1)
test=test.drop(features_drop, axis=1)

In [ ]:
train.head(2)

In [ ]:
test.head(2)

____

### Classification & Accuracy
    Define training and testing set

In [ ]:
#### train X= Exery thing except Weekly_Sales
train_X=train.drop(['Weekly_Sales','Date'], axis=1)

#### train Y= Only Weekly_Sales 
train_y=train['Weekly_Sales'] 
test_X=test.drop('Date',axis=1).copy()

train_X.shape, train_y.shape, test_X.shape

### Building models & comparing their RMSE values

#### 1.Linear Regression

In [ ]:
## Methood 1..
clf = LinearRegression()
clf.fit(train_X, train_y)
y_pred_linear=clf.predict(test_X)
acc_linear=round( clf.score(train_X, train_y) * 100, 2)
print ('scorbe:'+str(acc_linear) + ' percent')

#### 2. Random Forest

In [ ]:
clf = RandomForestRegressor(n_estimators=100)
clf.fit(train_X, train_y)
y_pred_rf=clf.predict(test_X)
acc_rf= round(clf.score(train_X, train_y) * 100, 2)
print ("Accuracy: %i %% \n"%acc_rf)

#### 3. Decision Tree

In [ ]:
clf=DecisionTreeRegressor()
clf.fit(train_X, train_y)
y_pred_dt= clf.predict(test_X)
acc_dt = round( clf.score(train_X, train_y) * 100, 2)
print (str(acc_dt) + ' percent')

#### Comparing Models
Let's compare the accuracy score of all the regression models used above.

In [ ]:
models = pd.DataFrame({
    'Model': ['Linear Regression','Random Forest','Decision Tree'],
    
    'Score': [acc_linear, acc_rf,acc_dt]
    })

models.sort_values(by='Score', ascending=False)

**Predicting Sales value for test data based on highest score model.**

In [ ]:
# Prediction value using Random Forest model..
submission = pd.DataFrame({
        "Store_Dept_Date": test.Store.astype(str)+'_'+test.Dept.astype(str)+'_'+test.Date.astype(str),
        "Weekly_Sales": y_pred_rf
    })

submission.to_csv('weekly_sales predicted.csv', index=False)
submission.to_excel(writer,'Weekly_sales Pred',index=False)

In [ ]:
submission.head()

    ************* THE END **************************************************************************************************